In [1]:
!pip install torch-fidelity
!pip install torchmetrics[image]

DEPRECATION: pytorch-lightning 1.7.6 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.7.6 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
generate_images_folder = './result'
orig_images_folder = './result'

gen_image_prefix = 'vae_'
orig_images_prefix = 'orig_'

In [4]:
import os
from PIL import Image
import torch
from torchvision import transforms
from torchmetrics.image.fid import FrechetInceptionDistance

# 이미지 폴더 경로
vae_images_folder = './result'
orig_images_folder = './result'

# 이미지 불러오기 및 텐서로 변환하는 함수
def load_images_from_folder(folder, start_dir):
    images = []
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # 이미지 크기를 128x128로 조정
        transforms.ToTensor(),
        transforms.Lambda(lambda x: (x * 255).byte())
    ])
    for filename in os.listdir(folder):
        if filename.startswith(start_dir):
            img_path = os.path.join(folder, filename)
            
            if not os.path.isfile(img_path):
                    continue
                    
            with Image.open(img_path) as img:                    
                img = transform(img)
                images.append(img) 
    return torch.stack(images)

# vae_로 시작하는 이미지 불러오기
vae_images = load_images_from_folder(generate_images_folder, gen_image_prefix)

# orig_로 시작하는 이미지 불러오기
orig_images = load_images_from_folder(orig_images_folder, orig_images_prefix)

# FID 인스턴스 생성
fid = FrechetInceptionDistance(feature=64)

# 실제(real) 이미지로 orig_ 이미지 업데이트
fid.update(orig_images, real=True)

# 생성된(generated) 이미지로 vae_ 이미지 업데이트
fid.update(vae_images, real=False)

# FID 계산
fid_value = fid.compute()


In [5]:
fid_value

tensor(5.1265)

In [6]:
def calculate_inception_score(images, batch_size=32, splits=10):
    # Inception 모델 로드
    inception_model = models.inception_v3(pretrained=True, transform_input=False)
    inception_model.eval()

    # 이미지 텐서를 batch_size로 나누기
    n_images = images.size(0)
    preds = []
    for i in range(0, n_images, batch_size):
        batch = images[i:i + batch_size].float() / 255.0  # 0~1 범위로 정규화
        with torch.no_grad():
            pred = inception_model(batch)
            preds.append(F.softmax(pred, dim=1).cpu())

    preds = torch.cat(preds, dim=0)

    # Inception Score 계산
    scores = []
    for i in range(splits):
        part = preds[i * (n_images // splits): (i + 1) * (n_images // splits), :]
        py = torch.mean(part, dim=0)
        scores.append(torch.exp(torch.mean(torch.sum(part * torch.log(part / py), dim=1))))

    return torch.mean(torch.tensor(scores)), torch.std(torch.tensor(scores))

In [8]:
import os
from PIL import Image
import torch
from torchvision import transforms, models
from torch.nn import functional as F

generated_images = load_images_from_folder(generate_images_folder, gen_image_prefix)

# Inception Score 계산
mean_score, std_score = calculate_inception_score(generated_images)

print(f"Inception Score: {mean_score.item()} ± {std_score.item()}")

/opt/conda/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

Inception Score: 3.3658289909362793 ± 1.016157627105713
